# Setup

On some systems, this cell needs to be run twice.

In [ ]:
%matplotlib inline
import importlib 
import numpy as np
import scipy as sp
from conservative_LMM import *
import matplotlib.pyplot as plt
from nodepy import rk
import time

# line cyclers adapted to colourblind people
from cycler import cycler
line_cycler   = (cycler(color=["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442", "#AB50B6"]) +
                 cycler(linestyle=["-", "--", "-.", ":", "-", "--", "-.", ":"]))
marker_cycler = (cycler(color=["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442", "#AB50B6"]) +
                 cycler(linestyle=["none", "none", "none", "none", "none", "none", "none", "none"]) +
                 cycler(marker=["4", "2", "3", "1", "+", "x", ".", (5, 2, 0)]))
marker_cycler6 = (cycler(color=["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7"]) +
                 cycler(linestyle=["none", "none", "none", "none", "none", "none"]) +
                 cycler(marker=["4", "2", "3", "1", "+", "x"]))

# matplotlib's standard cycler
standard_cycler = cycler("color", ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"])

plt.rc("axes", prop_cycle=line_cycler)

plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"\usepackage{newpxtext}\usepackage{newpxmath}\usepackage{commath}\usepackage{mathtools}")
plt.rc("font", family="serif", size=18.)
plt.rc("savefig", dpi=200)
plt.rc("legend", loc="best", fontsize="medium", fancybox=True, framealpha=0.5)
plt.rc("lines", linewidth=2.5, markersize=10, markeredgewidth=2.5)


# Nonlinear Oscillator

In [ ]:
def f_hr(u):
    return np.array([-u[1],u[0]]) / (u[0]*u[0]+u[1]*u[1])

def u_hr(t):
    return np.array([np.cos(t), np.sin(t)])


In [ ]:
t_final = 20
dts = np.array([0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0.001])

# Adams
fig_error, ax_error = plt.subplots(1, 1)
ax_error.set_prop_cycle(marker_cycler)
plt.xscale("log"); plt.yscale("log")
plt.xlabel(r"$\Delta t$")
plt.ylabel("Error")

error_b2, gammaM1_b2, error_p2, gammaM1_p2, error_rf2, gammaM1_rf2, error_rff2, gammaM1_rff2, error_ra2, gammaM1_ra2, error_idt2, gammaM1_idt2 = \
        compute_convergence_data(f_hr, u_hr, t_final, dts, conservative_AB2, 2, fixed_coefficients_twice=True)

error_b3, gammaM1_b3, error_p3, gammaM1_p3, error_rf3, gammaM1_rf3, error_rff3, gammaM1_rff3, error_ra3, gammaM1_ra3, error_idt3, gammaM1_idt3 = \
        compute_convergence_data(f_hr, u_hr, t_final, dts, conservative_AB3, 3, fixed_coefficients_twice=True)

ax_error.plot(dts, error_b2, label="Baseline, $k=2$")
ax_error.plot(dts, error_b3, label="Baseline, $k=3$")
ax_error.plot(dts, error_p2, label="Projection, $k=2$")
ax_error.plot(dts, error_p3, label="Projection, $k=3$")
ax_error.plot(dts, error_ra2, label="Relaxation (adaptive), $k=2$")
ax_error.plot(dts, error_ra3, label="Relaxation (adaptive), $k=3$")
ax_error.plot(dts, error_rff2, label="Relaxation (fixed), $k=2$")
ax_error.plot(dts, error_rff3, label="Relaxation (fixed), $k=3$")

plt.plot(dts, 10*dts**2, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^2)$", (2.e-3, 1.e-3), color="gray")
plt.plot(dts, 100*dts**3, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^3)$", (1.e-3, 1.e-8), color="gray")
plt.plot(dts, 10*dts**4, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^4)$", (4.e-3, 1.e-10), color="gray")

plt.ylim(1.0e-13, 1.0e2)
plt.savefig("../figures/convergence_nonlinear_osc_Adams.pdf", bbox_inches="tight")

# legend
plt.figure()
handles, labels = ax_error.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=4)
plt.savefig("../figures/convergence_nonlinear_osc_legend.pdf", bbox_inches="tight")

# Nyström
fig_error, ax_error = plt.subplots(1, 1)
ax_error.set_prop_cycle(marker_cycler)
plt.xscale("log"); plt.yscale("log")
plt.xlabel(r"$\Delta t$")
plt.ylabel("Error")

error_b2, gammaM1_b2, error_p2, gammaM1_p2, error_rf2, gammaM1_rf2, error_rff2, gammaM1_rff2, error_ra2, gammaM1_ra2, error_idt2, gammaM1_idt2 = \
        compute_convergence_data(f_hr, u_hr, t_final, dts, conservative_Nyström2AS, 2, 
                                 fixed_coefficients_twice=True, idx_u_old=-2)

error_b3, gammaM1_b3, error_p3, gammaM1_p3, error_rf3, gammaM1_rf3, error_rff3, gammaM1_rff3, error_ra3, gammaM1_ra3, error_idt3, gammaM1_idt3 = \
        compute_convergence_data(f_hr, u_hr, t_final, dts, conservative_Nyström3AS, 3, 
                                 fixed_coefficients_twice=True, idx_u_old=-2)

ax_error.plot(dts, error_b2)
ax_error.plot(dts, error_b3)
ax_error.plot(dts, error_p2)
ax_error.plot(dts, error_p3)
ax_error.plot(dts, error_ra2)
ax_error.plot(dts, error_ra3)
ax_error.plot(dts, error_rff2)
ax_error.plot(dts, error_rff3)

plt.plot(dts, 3*dts**2, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^2)$", (2.e-3, 1.e-6), color="gray")
plt.plot(dts[5:], 1.0e7*dts[5:]**3, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^3)$", (1.e-3, 1.0e-3), color="gray")
plt.plot(dts, 3*dts**4, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^4)$", (4.e-3, 5.e-11), color="gray")

plt.ylim(1.0e-13, 1.0e2)
plt.savefig("../figures/convergence_nonlinear_osc_Nystrom.pdf", bbox_inches="tight")

In [ ]:
dt = 0.01
t0 = 0.0; u0 = u_hr(t0)
t1 = t0 + dt; u1 = u_hr(t1)
t2 = t1 + dt; u2 = u_hr(t2)

time_tmp = time.time()
tt_b, uu_b, gamma_b = conservative_AB3(f_hr, t_final, t0, u0, t1, u1, t2, u2,
        return_gamma=True, projection=False, relaxation=False, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Baseline scheme:   %.2e s" % time_tmp)

time_tmp = time.time()
tt_ra, uu_ra, gamma_ra = conservative_AB3(f_hr, t_final, t0, u0, t1, u1, t2, u2,
        return_gamma=True, projection=False, relaxation=True, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Relaxation scheme: %.2e s" % time_tmp)

fig_eta, ax_eta = plt.subplots(1, 1)
plt.xlabel("$t$"); plt.ylabel(r"$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(0, t_final);
ax_eta.ticklabel_format(axis="y", style="sci", scilimits=(-2,2), useOffset=True)

ax_eta.plot(tt_b, [etaL2(u) for u in uu_b] - etaL2(uu_b[0]), label="Baseline")
ax_eta.plot(tt_ra, [etaL2(u) for u in uu_ra] - etaL2(uu_ra[0]), label="Relaxation")
ax_eta.set_yscale("symlog", linthreshy=1.0e-13)
ax_eta.legend()
fig_eta.savefig("../figures/nonlinear_osc_energy.pdf", bbox_inches="tight")

# Kepler Problem

In [ ]:
def f_kepler(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    abs_q = np.sqrt(q1*q1 + q2*q2)
    
    dq1 = p1
    dq2 = p2
    dp1 = -q1 / (abs_q*abs_q*abs_q)
    dp2 = -q2 / (abs_q*abs_q*abs_q)
    return np.array([dq1, dq2, dp1, dp2])

def kepler_energy(w):
    abs2_q = w[0]*w[0] + w[1]*w[1]
    abs2_p = w[2]*w[2] + w[3]*w[3]
    return 0.5 * abs2_p - 1.0 / np.sqrt(abs2_q)

def d_kepler_energy(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    abs_q = np.sqrt(q1*q1 + q2*q2)
    
    dq1 = q1 / (abs_q*abs_q*abs_q)
    dq2 = q2 / (abs_q*abs_q*abs_q)
    dp1 = p1
    dp2 = p2
    return np.array([dq1, dq2, dp1, dp2])

def kepler_angular_momentum(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    
    return q1*p2 - q2*p1

def d_kepler_angular_momentum(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    
    return np.array([p2, -p1, -q2, q1])

"""
Analytical solution of the Kepler problem, cf.
http://mathworld.wolfram.com/KeplersEquation.html
and
https://matlab-monkey.com/astro/keplerEquation/KeplerEquationPub.html
"""
def u_kepler(t):
    e = 0.5
    u0 = np.array([1.0 - e, 0.0, 0.0, np.sqrt((1+e)/(1-e))])
    energy = 0.5 * (u0[2]*u0[2] + u0[3]*u0[3]) - 1.0 / np.sqrt(u0[0]*u0[0] + u0[1]*u0[1])
    momentum = u0[0]*u0[3] - u0[1]*u0[2]
    
    res = fsolve(lambda alpha: alpha - e*np.sin(alpha) - t, 0, xtol=1.e-12)
    alpha = res[0]
    theta = 2.0 * np.arctan(np.sqrt((1+e)/(1-e)) * np.tan(0.5*alpha))
    radius = (1 - e*e) / (1 + e * np.cos(theta))
    q0 = radius * np.cos(theta)
    q1 = radius * np.sin(theta)
    
    # using the conservation of angular momentum and energy
    abs2p = 2 * (energy + 1.0 / np.sqrt(q0*q0 + q1*q1))
    if q1 <= 0:
        p0 = -((momentum*q1 + np.sqrt(q0**2 * (-momentum**2 + abs2p*(q0**2 + q1**2)))) / (q0**2 + q1**2))
        p1 = (momentum*q0**2 - q1*np.sqrt(q0**2 * (-momentum**2 + abs2p*q0**2 + abs2p*q1**2))) / (q0**3 + q0*q1**2)
    else:
        p0 = (-(momentum*q1) + np.sqrt(q0**2 * (-momentum**2 + abs2p*(q0**2 + q1**2)))) / (q0**2 + q1**2)
        p1 = (momentum*q0**2 + q1 * np.sqrt(q0**2 * (-momentum**2 + abs2p*q0**2 + abs2p*q1**2))) / (q0**3 + q0*q1**2)
    return np.array([q0, q1, p0, p1])


In [ ]:
t_final = 5
dts = np.array([0.1, 0.05, 0.05, 0.01, 0.005, 0.0025, 0.001, 0.0005, 0.00025])

# eBDF
fig_error, ax_error = plt.subplots(1, 1)
ax_error.set_prop_cycle(marker_cycler6)
plt.xscale("log"); plt.yscale("log")
plt.xlabel(r"$\Delta t$")
plt.ylabel("Error")

error_b3, _, error_p3, _, error_rf3, _, error_ra3, _, error_idt3, gammaM1_idt2 = \
        compute_convergence_data(f_kepler, u_kepler, t_final, dts, conservative_eBDF3, 3, 
                                 eta=kepler_energy, deta=d_kepler_energy, error_idx=np.array([0,1]))

error_b4, _, error_p4, _, error_rf4, _, error_ra4, _, error_idt4, _ = \
        compute_convergence_data(f_kepler, u_kepler, t_final, dts, conservative_eBDF4, 4,
                                 eta=kepler_energy, deta=d_kepler_energy, error_idx=np.array([0,1]))

ax_error.plot(dts, error_b3, label="Baseline, $p=k=3$")
ax_error.plot(dts, error_p3, label="Projection, $p=k=3$")
ax_error.plot(dts, error_ra3, label="Relaxation, $p=k=3$")
ax_error.plot(dts, error_b4, label="Baseline, $p=k=4$")
ax_error.plot(dts, error_p4, label="Projection, $p=k=4$")
ax_error.plot(dts, error_ra4, label="Relaxation, $p=k=4$")

plt.plot(dts, 1.0e3*dts**3, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^3)$", (1.e-3, 1.e-4), color="gray")
plt.plot(dts, 2.0e2*dts**4, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^4)$", (3.e-3, 1.e-9), color="gray")

plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.savefig("../figures/convergence_kepler_eBDF.pdf", bbox_inches="tight")

In [ ]:
dt = 0.01
t0 = 0.0; u0 = u_kepler(t0)
t1 = t0 + dt; u1 = u_kepler(t1)
t2 = t1 + dt; u2 = u_kepler(t2)

time_tmp = time.time()
tt_b, uu_b, gamma_b = conservative_AB3(f_kepler, t_final, t0, u0, t1, u1, t2, u2,
        eta=kepler_energy, deta=d_kepler_energy,
        return_gamma=True, projection=False, relaxation=False, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Baseline scheme:   %.2e s" % time_tmp)

time_tmp = time.time()
tt_ra, uu_ra, gamma_ra = conservative_AB3(f_kepler, t_final, t0, u0, t1, u1, t2, u2,
        eta=kepler_energy, deta=d_kepler_energy,
        return_gamma=True, projection=False, relaxation=True, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Relaxation scheme: %.2e s" % time_tmp)

fig_eta, ax_eta = plt.subplots(1, 1)
plt.xlabel("$t$"); plt.ylabel(r"$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(0, t_final);
ax_eta.ticklabel_format(axis="y", style="sci", scilimits=(-2,2), useOffset=True)

ax_eta.plot(tt_b, [kepler_energy(u) for u in uu_b] - kepler_energy(uu_b[0]), label="Baseline")
ax_eta.plot(tt_ra, [kepler_energy(u) for u in uu_ra] - kepler_energy(uu_ra[0]), label="Relaxation")
ax_eta.set_yscale("symlog", linthreshy=1.0e-13)
ax_eta.legend()
fig_eta.savefig("../figures/kepler_energy.pdf", bbox_inches="tight")

# Dissipated Exponential Entropy

In [ ]:
def f_diss_exp(w):
    return -np.exp(w)

def eta_diss_exp(w):
    return np.exp(w)

def d_eta_diss_exp(w):
    return np.exp(w)

def u_diss_exp(t):
    # initial condition 0.5
    w = -np.log( np.exp(-0.5) + t )
    return w


In [ ]:
t_final = 5
dts = np.array([0.1, 0.05, 0.05, 0.01, 0.005, 0.0025, 0.001, 0.0005, 0.00025])


fig_error_b, ax_error_b = plt.subplots(1, 1)
ax_error_b.set_prop_cycle(marker_cycler)
plt.xscale("log"); plt.yscale("log")
plt.xlabel(r"$\Delta t$")
plt.ylabel("Error")

fig_error_p, ax_error_p = plt.subplots(1, 1)
ax_error_p.set_prop_cycle(marker_cycler)
plt.xscale("log"); plt.yscale("log")
plt.xlabel(r"$\Delta t$")
plt.ylabel("Error")
fig_gamma_p, ax_gamma_p = plt.subplots(1, 1)
ax_gamma_p.set_prop_cycle(marker_cycler)
plt.xscale("log"); plt.yscale("symlog", linthreshy=1.0e-12)
plt.xlabel(r"$\Delta t$")
plt.ylabel(r"$\| \gamma - 1 \|_\infty$")

fig_error_ra, ax_error_ra = plt.subplots(1, 1)
ax_error_ra.set_prop_cycle(marker_cycler)
plt.xscale("log"); plt.yscale("log")
plt.xlabel(r"$\Delta t$")
plt.ylabel("Error")
fig_gamma_ra, ax_gamma_ra = plt.subplots(1, 1)
ax_gamma_ra.set_prop_cycle(marker_cycler)
plt.xscale("log"); plt.yscale("symlog", linthreshy=1.0e-12)
plt.xlabel(r"$\Delta t$")
plt.ylabel(r"$\| \gamma - 1 \|_\infty$")

error_b, gammaM1_b, error_p, gammaM1_p, error_rf, gammaM1_rf, error_ra, gammaM1_ra, error_idt, gammaM1_idt = \
    compute_convergence_data(f_diss_exp, u_diss_exp, t_final, dts, cons_or_diss_AB3, 3, 
                             eta=eta_diss_exp, deta=d_eta_diss_exp)
label = "Adams($3$)"
ax_error_b.plot(dts, error_b, label=label)
ax_error_p.plot(dts, error_p, label=label)
ax_gamma_p.plot(dts, gammaM1_p, label=label)
ax_error_ra.plot(dts, error_ra, label=label)
ax_gamma_ra.plot(dts, gammaM1_ra, label=label)

error_b, gammaM1_b, error_p, gammaM1_p, error_rf, gammaM1_rf, error_ra, gammaM1_ra, error_idt, gammaM1_idt = \
    compute_convergence_data(f_diss_exp, u_diss_exp, t_final, dts, cons_or_diss_AB4, 4, 
                             eta=eta_diss_exp, deta=d_eta_diss_exp)
label = "Adams($4$)"
ax_error_b.plot(dts, error_b, label=label)
ax_error_p.plot(dts, error_p, label=label)
ax_gamma_p.plot(dts, gammaM1_p, label=label)
ax_error_ra.plot(dts, error_ra, label=label)
ax_gamma_ra.plot(dts, gammaM1_ra, label=label)

error_b, gammaM1_b, error_p, gammaM1_p, error_rf, gammaM1_rf, error_ra, gammaM1_ra, error_idt, gammaM1_idt = \
    compute_convergence_data(f_diss_exp, u_diss_exp, t_final, dts, cons_or_diss_eBDF3AS, 3, 
                             eta=eta_diss_exp, deta=d_eta_diss_exp)
label = "eBDF($3$)AS"
ax_error_b.plot(dts, error_b, label=label)
ax_error_p.plot(dts, error_p, label=label)
ax_gamma_p.plot(dts, gammaM1_p, label=label)
ax_error_ra.plot(dts, error_ra, label=label)
ax_gamma_ra.plot(dts, gammaM1_ra, label=label)

error_b, gammaM1_b, error_p, gammaM1_p, error_rf, gammaM1_rf, error_ra, gammaM1_ra, error_idt, gammaM1_idt = \
    compute_convergence_data(f_diss_exp, u_diss_exp, t_final, dts, cons_or_diss_EDC22, 3, 
                             eta=eta_diss_exp, deta=d_eta_diss_exp)
label = "EDC($2,2$)"
ax_error_b.plot(dts, error_b, label=label)
ax_error_p.plot(dts, error_p, label=label)
ax_gamma_p.plot(dts, gammaM1_p, label=label)
ax_error_ra.plot(dts, error_ra, label=label)
ax_gamma_ra.plot(dts, gammaM1_ra, label=label)

error_b, gammaM1_b, error_p, gammaM1_p, error_rf, gammaM1_rf, error_ra, gammaM1_ra, error_idt, gammaM1_idt = \
    compute_convergence_data(f_diss_exp, u_diss_exp, t_final, dts, cons_or_diss_EDC23, 4, 
                             eta=eta_diss_exp, deta=d_eta_diss_exp)
label = "EDC($2,3$)"
ax_error_b.plot(dts, error_b, label=label)
ax_error_p.plot(dts, error_p, label=label)
ax_gamma_p.plot(dts, gammaM1_p, label=label)
ax_error_ra.plot(dts, error_ra, label=label)
ax_gamma_ra.plot(dts, gammaM1_ra, label=label)

error_b, gammaM1_b, error_p, gammaM1_p, error_rf, gammaM1_rf, error_ra, gammaM1_ra, error_idt, gammaM1_idt = \
    compute_convergence_data(f_diss_exp, u_diss_exp, t_final, dts, cons_or_diss_EDC33, 4, 
                             eta=eta_diss_exp, deta=d_eta_diss_exp)
label = "EDC($3,3$)"
ax_error_b.plot(dts, error_b, label=label)
ax_error_p.plot(dts, error_p, label=label)
ax_gamma_p.plot(dts, gammaM1_p, label=label)
ax_error_ra.plot(dts, error_ra, label=label)
ax_gamma_ra.plot(dts, gammaM1_ra, label=label)

error_b, gammaM1_b, error_p, gammaM1_p, error_rf, gammaM1_rf, error_ra, gammaM1_ra, error_idt, gammaM1_idt = \
    compute_convergence_data(f_diss_exp, u_diss_exp, t_final, dts, cons_or_diss_SSP43, 4, 
                             eta=eta_diss_exp, deta=d_eta_diss_exp)
label = "SSP($4,3$)"
ax_error_b.plot(dts, error_b, label=label)
ax_error_p.plot(dts, error_p, label=label)
ax_gamma_p.plot(dts, gammaM1_p, label=label)
ax_error_ra.plot(dts, error_ra, label=label)
ax_gamma_ra.plot(dts, gammaM1_ra, label=label)

error_b, gammaM1_b, error_p, gammaM1_p, error_rf, gammaM1_rf, error_ra, gammaM1_ra, error_idt, gammaM1_idt = \
    compute_convergence_data(f_diss_exp, u_diss_exp, t_final, dts, cons_or_diss_SSP43AS, 4, 
                             eta=eta_diss_exp, deta=d_eta_diss_exp)
label = "SSP($4,3$)AS"
ax_error_b.plot(dts, error_b, label=label)
ax_error_p.plot(dts, error_p, label=label)
ax_gamma_p.plot(dts, gammaM1_p, label=label)
ax_error_ra.plot(dts, error_ra, label=label)
ax_gamma_ra.plot(dts, gammaM1_ra, label=label)

# legend
plt.figure()
handles, labels = ax_error_b.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=4)
plt.savefig("../figures/convergence_diss_exp_legend.pdf", bbox_inches="tight")

plt.figure(fig_error_b.number)
plt.ylim(1.e-15, 1.e-2)
plt.plot(dts, 1*dts**3, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^3)$", (1.e-3, 1.e-7), color="gray")
plt.plot(dts, 2*dts**4, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^4)$", (3.e-4, 3.e-12), color="gray")
plt.savefig("../figures/convergence_diss_exp_error_b.pdf", bbox_inches="tight")

plt.figure(fig_error_p.number)
plt.ylim(1.e-15, 1.e-2)
plt.plot(dts, 1*dts**3, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^3)$", (1.e-3, 1.e-7), color="gray")
plt.plot(dts, 0.5*dts**4, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^4)$", (3.e-4, 1.e-12), color="gray")
plt.plot(dts, 0.5*dts**5, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^5)$", (8.e-3, 1.e-12), color="gray")
plt.savefig("../figures/convergence_diss_exp_error_p.pdf", bbox_inches="tight")

plt.figure(fig_error_ra.number)
plt.ylim(1.e-15, 1.e-2)
plt.plot(dts, 10*dts**3, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^3)$", (1.e-3, 1.e-6), color="gray")
plt.plot(dts, 0.5*dts**4, marker="None", linestyle=":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^4)$", (2.e-3, 1.e-12), color="gray")
plt.savefig("../figures/convergence_diss_exp_error_ra.pdf", bbox_inches="tight")

In [ ]:
def compute(dt):
    t_final = 5
    t0 = 0*dt; u0 = u_diss_exp(t0)
    t1 = 1*dt; u1 = u_diss_exp(t1)
    t2 = 2*dt; u2 = u_diss_exp(t2)
    t3 = 3*dt; u3 = u_diss_exp(t3)
    
    tt_SSP32, uu_SSP32, gamma_SSP32 = cons_or_diss_SSP32(f_diss_exp, t_final, t0, u0, t1, u1, t2, u2,
                                        idx_u_old=adaptive_u_old_SSP32,
                                        eta=eta_diss_exp, deta=d_eta_diss_exp,
                                        projection=False, relaxation=True, adapt_dt=True, 
                                        adapt_coefficients=False, fixed_coefficient_fix=True,
                                        return_gamma=True)
    tau_SSP32 = dt*np.arange(len(tt_SSP32))
    tt_SSP43, uu_SSP43, gamma_SSP43 = cons_or_diss_SSP43(f_diss_exp, t_final, t0, u0, t1, u1, t2, u2, t3, u3,
                                        idx_u_old=adaptive_u_old_SSP43,
                                        eta=eta_diss_exp, deta=d_eta_diss_exp,
                                        projection=False, relaxation=True, adapt_dt=True, 
                                        adapt_coefficients=False, fixed_coefficient_fix=True,
                                        return_gamma=True)
    tau_SSP43 = dt*np.arange(len(tt_SSP43))
     
    return tau_SSP32, tt_SSP32, gamma_SSP32, tau_SSP43, tt_SSP43, gamma_SSP43


fig, ax = plt.subplots(1, 1)
ax.ticklabel_format(axis="y", style="sci", scilimits=(-1,1))
ax.set_xscale("symlog", linthreshx=1.0e-4)
ax.set_yscale("symlog", linthreshy=1.0e-12)
tau_SSP32_1, tt_SSP32_1, _, tau_SSP43_1, tt_SSP43_1, _ = compute(1.0e-2)
tau_SSP32_2, tt_SSP32_2, _, tau_SSP43_2, tt_SSP43_2, _ = compute(1.0e-4)
plt.plot(tau_SSP32_1, tt_SSP32_1-tau_SSP32_1, label=r"SSP($3, 2$), $\Delta \tau = 10^{-2}$")
plt.plot(tau_SSP32_2, tt_SSP32_2-tau_SSP32_2, label=r"SSP($3, 2$), $\Delta \tau = 10^{-4}$")
plt.plot(tau_SSP43_1, tt_SSP43_1-tau_SSP43_1, label=r"SSP($4, 3$), $\Delta \tau = 10^{-2}$")
print("%.2e"%(tt_SSP43_1[-1]-tau_SSP43_1[-1]))
plt.plot(tau_SSP43_2, tt_SSP43_2-tau_SSP43_2, label=r"SSP($4, 3$), $\Delta \tau = 10^{-4}$")
print("%.2e"%(tt_SSP43_2[-1]-tau_SSP43_2[-1]))
plt.locator_params(axis="y", numticks=11)
labels = ax.get_yticklabels(); plt.setp(labels[len(labels)//2], visible=False)
plt.xlabel(r"$\tau$")
plt.ylabel(r"$t - \tau$")
plt.xlim(1.0e-4, t_final)
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));
plt.savefig("../figures/t_tau_diss_exp.pdf", bbox_inches="tight")

# Euler Equations

In [ ]:
N = 100

xx, dx = np.linspace(0, 2, N+1, endpoint=True, retstep=True)
x = xx[:-1] 

GAMMA = 1.4

def u_analytical(t):
    rho0 = 1.0 + 0.5 * np.sin(np.pi * (x - t))
    v0 = np.ones(N)
    p0 = np.ones(N)
    rho_v0 = rho0 * v0
    rho_e0 = p0 / (GAMMA - 1.0) + 0.5 * rho0 * v0*v0
    u0 = np.zeros((5, N))
    u0[0,:] = rho0
    u0[1,:] = rho_v0
    u0[2,:] = rho_e0
    u0[3,:] = v0
    u0[4,:] = p0
    u0 = np.reshape(u0, np.size(u0))
    return u0

u0 = u_analytical(0.0)

def logmean(aa, bb):
    a = np.minimum(aa, bb)
    b = np.maximum(aa, bb)
    zeta = a / b
    f = (zeta-1) / (zeta+1)
    u = f*f
    value1 = 1 + u * (1./3. + u * (1./5. + u * (1./7.)))
    value2 = np.log(zeta)/(2.0 * f)
    F = np.zeros_like(u)
    F[u < 1.0e-2] = value1[u < 1.0e-2]
    F[u >= 1.0e-2] = value2[u >= 1.0e-2]
    return (a+b)/(2.0 * F)

def fnum(fluxes, rho_l, v_l, p_l, rho_r, v_r, p_r):
    beta_l = 0.5 * rho_l / p_l
    beta_r = 0.5 * rho_r / p_r
    
    rho      = 0.5 * (rho_l + rho_r)
    rho_log  = logmean(rho_l, rho_r)
    vx       = 0.5 * (v_l + v_r) 
    v2       = 0.5 * (v_l*v_l + v_r*v_r) 
    beta     = 0.5 * (beta_l + beta_r) 
    beta_log = logmean(beta_l, beta_r)
    diff_correction = 0.25 * (p_l - p_r) * (v_l - v_r)
    
    fluxes[0, :] = rho_log * vx
    fluxes[1, :] = vx * fluxes[0, :] + 0.5 * (p_l + p_r)
    fluxes[2, :] = 1./(2*GAMMA-2) * fluxes[0, :] / beta_log - 0.5 * v2 * fluxes[0, :] + vx * fluxes[1, :] - diff_correction
    
    return

def f_euler(uu):
    u = np.reshape(uu, (5, N))
    
    # compute primitive variables
    u[3,:] = u[1,:] / u[0,:] # velocity
    u[4,:] = (GAMMA - 1.0) * (u[2,:] - 0.5 * u[1,:] * u[3,:]) # pressure
    
    # left and right values
    ul = np.zeros((5, N+1))
    ul[:, 1:] = u; ul[:, 0] = u[:, -1]
    ur = np.zeros_like(ul)
    ur[:, :-1] = u; ur[:, -1] = u[:, 0]
    
    # compute numerical fluxes
    fluxes = np.zeros_like(ul)
    fnum(fluxes, ul[0,:], ul[3,:], ul[4,:], ur[0,:], ur[3,:], ur[4,:])
    
    # compute flux differences
    fluxdiff = -(fluxes[:, 1:] - fluxes[:, :-1]) / dx
    
    return np.reshape(fluxdiff, uu.shape)

def eta_euler(uu):
    u = np.reshape(uu, (5, N))
    
    # compute primitive variables
    u[3,:] = u[1,:] / u[0,:] # velocity
    u[4,:] = (GAMMA - 1.0) * (u[2,:] - 0.5 * u[1,:] * u[3,:]) # pressure
    
    return np.sum(- u[0, :] * (np.log(u[4,:]) - GAMMA * np.log(u[0,:])) / (GAMMA - 1.0)) * dx

def deta_euler(uu):
    u = np.reshape(uu, (5, N))
    
    # compute primitive variables
    u[3,:] = u[1,:] / u[0,:] # velocity
    u[4,:] = (GAMMA - 1.0) * (u[2,:] - 0.5 * u[1,:] * u[3,:]) # pressure
    
    w = np.zeros_like(u)
    w[0,:] = (GAMMA -  (np.log(u[4,:]) - GAMMA * np.log(u[0,:]))) / (GAMMA - 1.0) - 0.5 * u[1,:]*u[3,:] / u[4,:]
    w[1,:] = u[1,:] / u[4,:]
    w[2,:] = -u[0,:] / u[4,:]
    
    return np.reshape(w * dx, np.shape(uu)) 

cfl = 0.1
dt = cfl*dx
t_final = 50.

ssp33 = rk.loadRKM("SSP33")
tt0, uu0 = relaxation_ERK(ssp33, dt, f_euler, eta_euler, deta_euler, u0, 3, 
                          relaxed=True, method="brentq", newdt=True, tol=1.e-14)
t0 = tt0[0]; u0 = uu0[:,0]
t1 = tt0[1]; u1 = uu0[:,1]
t2 = tt0[2]; u2 = uu0[:,2]
t3 = tt0[3]; u3 = uu0[:,3]


time_tmp = time.time()
tt_b, uu_b, gamma_b = conservative_SSP43(f_euler, t_final, t0, u0, t1, u1, t2, u2, t3, u3,
        eta=eta_euler, deta=deta_euler,
        return_gamma=True, projection=False, relaxation=False, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Baseline scheme:   %.2e s" % time_tmp)

time_tmp = time.time()
tt_p, uu_p, gamma_p = conservative_SSP43(f_euler, t_final, t0, u0, t1, u1, t2, u2, t3, u3,
        eta=eta_euler, deta=deta_euler,
        return_gamma=True, projection=True, relaxation=False, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Projection scheme: %.2e s" % time_tmp)

time_tmp = time.time()
tt_ra, uu_ra, gamma_ra = conservative_SSP43(f_euler, t_final, t0, u0, t1, u1, t2, u2, t3, u3,
        eta=eta_euler, deta=deta_euler, method="brentq",
        return_gamma=True, projection=False, relaxation=True, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Relaxation scheme: %.2e s" % time_tmp)

In [ ]:
fig_solution, ax_solution = plt.subplots(1, 1)
plt.xlabel("$x$"); plt.ylabel(r"Density $\varrho$"); plt.xlim(xx[0], xx[-1]);
fig_eta, ax_eta = plt.subplots(1, 1)
plt.xlabel("$t$"); plt.ylabel(r"$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(0, t_final);
ax_eta.ticklabel_format(axis="y", style="sci", scilimits=(-2,2), useOffset=True)
fig_mass, ax_mass = plt.subplots(1, 1)
plt.xlabel("$t$"); plt.ylabel(r"Total Mass"); plt.xlim(0, t_final);
ax_mass.ticklabel_format(axis="y", style="sci", scilimits=(-2,2), useOffset=True)
fig_error, ax_error = plt.subplots(1, 1)
plt.xlabel("$t$"); plt.ylabel(r"Error"); plt.xlim(0, t_final);
ax_error.ticklabel_format(axis="y", style="sci", scilimits=(-2,2), useOffset=True)

tt = tt_b; uu = uu_b
u = np.reshape(uu[-1], (5,N))
H = [eta_euler(v) for v in uu]
mass = [np.sum(np.reshape(v, (5,N))[0,:]) * dx for v in uu]
error = [np.sqrt(np.sum(np.linalg.norm(uu[i] - u_analytical(tt[i]))**2)) * dx for i in np.arange(len(uu))]
plt.figure(fig_solution.number)
plt.plot(xx, np.hstack((u[0,:], u[0,0])), label="Baseline")
plt.figure(fig_eta.number)
plt.plot(tt, H - H[0])
plt.figure(fig_mass.number)
plt.plot(tt, mass)
plt.figure(fig_error.number)
plt.plot(tt, error)

tt = tt_p; uu = uu_p
u = np.reshape(uu[-1], (5,N))
H = [eta_euler(v) for v in uu]
mass = [np.sum(np.reshape(v, (5,N))[0,:]) * dx for v in uu]
error = [np.sqrt(np.sum(np.linalg.norm(uu[i] - u_analytical(tt[i]))**2)) * dx for i in np.arange(len(uu))]
plt.figure(fig_solution.number)
plt.plot(xx, np.hstack((u[0,:], u[0,0])), label="Projection")
plt.figure(fig_eta.number)
plt.plot(tt, H - H[0])
plt.figure(fig_mass.number)
plt.plot(tt, mass)
plt.figure(fig_error.number)
plt.plot(tt, error)

tt = tt_ra; uu = uu_ra
u = np.reshape(uu[-1], (5,N))
H = [eta_euler(v) for v in uu]
mass = [np.sum(np.reshape(v, (5,N))[0,:]) * dx for v in uu]
error = [np.sqrt(np.sum(np.linalg.norm(uu[i] - u_analytical(tt[i]))**2)) * dx for i in np.arange(len(uu))]
plt.figure(fig_solution.number)
plt.plot(xx, np.hstack((u[0,:], u[0,0])), label="Relaxation")
# plt.savefig("../figures/euler_solution.pdf", bbox_inches="tight")
plt.figure(fig_eta.number)
plt.plot(tt, H - H[0])
plt.yscale("symlog", linthreshy=1.0e-13)
plt.savefig("../figures/euler_eta.pdf", bbox_inches="tight")
plt.figure(fig_mass.number)
plt.plot(tt, mass)
plt.savefig("../figures/euler_mass.pdf", bbox_inches="tight")
plt.figure(fig_error.number)
plt.plot(tt, error)
# plt.savefig("../figures/euler_error.pdf", bbox_inches="tight")

plt.figure()
handles, labels = ax_solution.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=4)
plt.savefig("../figures/euler_legend.pdf", bbox_inches="tight")

# Burgers' Equation

In [ ]:
def fnum_burgers_EC_diss(fluxes, u_l, u_r):
    eps = 0.01
    fluxes[:] = (u_l*u_l + u_l*u_r + u_r*u_r) / 6. - eps * (u_r - u_l)
    return

def fnum_burgers_central_diss(fluxes, u_l, u_r):
    eps = 0.01
    fluxes[:] = (u_l*u_l + u_r*u_r) / 4. - eps * (u_r - u_l)
    return

def f_fnum_burgers(u, fnum):
    # left and right values
    ul = np.zeros(len(u)+1)
    ul[1:] = u; ul[0] = u[-1]
    ur = np.zeros_like(ul)
    ur[:-1] = u; ur[-1] = u[0]
    
    # compute numerical fluxes
    fluxes = np.zeros_like(ul)
    fnum(fluxes, ul, ur)
    
    # compute flux differences
    fluxdiff = -(fluxes[1:] - fluxes[:-1]) / dx
    return fluxdiff

def eta_burgers(uu):
    return etaL2(uu) * dx

def deta_burgers(uu):
    return uu * dx


def solve_and_save_burgers(fnum, figname):
    N = 50
    xx, dx = np.linspace(-1, 1, N+1, endpoint=True, retstep=True)
    x = xx[:-1] 

    u0 = np.exp(-30*x**2)
    cfl = 0.2
    dt = cfl*dx
    t_final = 0.25
    
    f_burgers = lambda u: f_fnum_burgers(u, fnum)

    tt0, uu0 = relaxation_ERK(rk.loadRKM("SSP33"), dt, f_burgers, eta_burgers, deta_burgers, u0, 3, 
                              relaxed=False, method="brentq", newdt=True, tol=1.e-14)
    t0 = tt0[0]; u0 = uu0[:,0]
    t1 = tt0[1]; u1 = uu0[:,1]
    t2 = tt0[2]; u2 = uu0[:,2]
    t3 = tt0[3]; u3 = uu0[:,3]

    time_tmp = time.time()
    tt_SSP32_b, uu_SSP32_b, gamma_SSP32_b = cons_or_diss_SSP32(f_burgers, t_final, t0, u0, t1, u1, t2, u2,
            eta=etaL2, deta=detaL2,
            return_gamma=True, projection=False, relaxation=False, adapt_dt=True, adapt_coefficients=True,
            idx_u_old=-1)
    time_tmp = time.time() - time_tmp
    print("Baseline scheme:   %.2e s" % time_tmp)

    time_tmp = time.time()
    tt_SSP32_p, uu_SSP32_p, gamma_SSP32_p = cons_or_diss_SSP32(f_burgers, t_final, t0, u0, t1, u1, t2, u2,
            eta=etaL2, deta=detaL2,
            return_gamma=True, projection=True, relaxation=False, adapt_dt=True, adapt_coefficients=True,
            idx_u_old=-1)
    time_tmp = time.time() - time_tmp
    print("Projection scheme:   %.2e s" % time_tmp)

    time_tmp = time.time()
    tt_SSP32_r, uu_SSP32_r, gamma_SSP32_r = cons_or_diss_SSP32(f_burgers, t_final, t0, u0, t1, u1, t2, u2,
            eta=etaL2, deta=detaL2,
            return_gamma=True, projection=False, relaxation=True, adapt_dt=True, adapt_coefficients=True,
            idx_u_old=-1)
    time_tmp = time.time() - time_tmp
    print("Relaxation scheme:   %.2e s" % time_tmp)


    time_tmp = time.time()
    tt_SSP43_b, uu_SSP43_b, gamma_SSP43_b = cons_or_diss_SSP43(f_burgers, t_final, t0, u0, t1, u1, t2, u2, t3, u3,
            eta=etaL2, deta=detaL2,
            return_gamma=True, projection=False, relaxation=False, adapt_dt=True, adapt_coefficients=True,
            idx_u_old=adaptive_u_old_SSP43)
    time_tmp = time.time() - time_tmp
    print("Baseline scheme:   %.2e s" % time_tmp)

    time_tmp = time.time()
    tt_SSP43_p, uu_SSP43_p, gamma_SSP43_p = cons_or_diss_SSP43(f_burgers, t_final, t0, u0, t1, u1, t2, u2, t3, u3,
            eta=etaL2, deta=detaL2,
            return_gamma=True, projection=True, relaxation=False, adapt_dt=True, adapt_coefficients=True,
            idx_u_old=adaptive_u_old_SSP43)
    time_tmp = time.time() - time_tmp
    print("Projection scheme:   %.2e s" % time_tmp)

    time_tmp = time.time()
    tt_SSP43_r, uu_SSP43_r, gamma_SSP43_r = cons_or_diss_SSP43(f_burgers, t_final, t0, u0, t1, u1, t2, u2, t3, u3,
            eta=etaL2, deta=detaL2,
            return_gamma=True, projection=False, relaxation=True, adapt_dt=True, adapt_coefficients=True,
            idx_u_old=adaptive_u_old_SSP43)
    time_tmp = time.time() - time_tmp
    print("Relaxation scheme:   %.2e s" % time_tmp)

    time_tmp = time.time()
    dt_ref = 0.01 * dt
    tt_ref, uu_ref = relaxation_ERK(rk.loadRKM("PD8"), dt_ref, f_burgers, eta_burgers, deta_burgers, u0, t_final/dt_ref, 
                              relaxed=False, method="brentq", newdt=True, tol=1.e-14)
    time_tmp = time.time() - time_tmp
    print("Reference scheme:   %.2e s\n" % time_tmp)
    
    
    fig_eta, ax_eta = plt.subplots(1, 1)
    plt.xlabel("$t$"); plt.ylabel(r"$\eta(u_\mathrm{num}(t)) - \eta(u_0)$"); plt.xlim(0, t_final);
    ax_eta.ticklabel_format(axis="y", style="sci", scilimits=(-2,2), useOffset=True)

    tt = tt_SSP32_b; uu = uu_SSP32_b
    H = [eta_burgers(v) for v in uu]
    plt.plot(tt, (H - H[0]), label="Baseline SSP($3, 2$)")
    tt = tt_SSP43_b; uu = uu_SSP43_b
    H = [eta_burgers(v) for v in uu]
    plt.plot(tt, (H - H[0]), label="Baseline SSP($4, 3$)")
    tt = tt_SSP32_p; uu = uu_SSP32_p
    H = [eta_burgers(v) for v in uu]
    plt.plot(tt, (H - H[0]), label="Projection SSP($3, 2$)")
    tt = tt_SSP43_p; uu = uu_SSP43_p
    H = [eta_burgers(v) for v in uu]
    plt.plot(tt, (H - H[0]), label="Projection SSP($4, 3$)")
    tt = tt_SSP32_r; uu = uu_SSP32_r
    H = [eta_burgers(v) for v in uu]
    plt.plot(tt, (H - H[0]), label="Relaxation SSP($3, 2$)")
    tt = tt_SSP43_r; uu = uu_SSP43_r
    H = [eta_burgers(v) for v in uu]
    plt.plot(tt, (H - H[0]), label="Relaxation SSP($4, 3$)")

    tt = tt_ref; uu = [uu_ref[:,i] for i in np.arange(uu_ref.shape[1])]
    H = [eta_burgers(v) for v in uu]
    plt.plot(tt, (H - H[0]), label="Reference")
    plt.savefig("../figures/burgers_energy_%s.pdf"%(figname), bbox_inches="tight")

    
    plt.figure()
    handles, labels = ax_eta.get_legend_handles_labels()
    plt.figlegend(handles, labels, loc="center", ncol=4)
    plt.savefig("../figures/burgers_legend.pdf", bbox_inches="tight")

    
    fig_mass, ax_mass = plt.subplots(1, 1)
    plt.xlabel("$t$"); plt.ylabel(r"Change of Total Mass"); plt.xlim(0, t_final);
    ax_mass.ticklabel_format(axis="y", style="sci", scilimits=(-2,2), useOffset=True)

    tt = tt_SSP32_b; uu = uu_SSP32_b
    mass = [np.sum(v) * dx for v in uu]
    plt.plot(tt, mass - mass[0])
    tt = tt_SSP43_b; uu = uu_SSP43_b
    mass = [np.sum(v) * dx for v in uu]
    plt.plot(tt, mass - mass[0])
    tt = tt_SSP32_p; uu = uu_SSP32_p
    mass = [np.sum(v) * dx for v in uu]
    plt.plot(tt, mass - mass[0])
    tt = tt_SSP43_p; uu = uu_SSP43_p
    mass = [np.sum(v) * dx for v in uu]
    plt.plot(tt, mass - mass[0])
    tt = tt_SSP32_r; uu = uu_SSP32_r
    mass = [np.sum(v) * dx for v in uu]
    plt.plot(tt, mass - mass[0])
    tt = tt_SSP43_r; uu = uu_SSP43_r
    mass = [np.sum(v) * dx for v in uu]
    plt.plot(tt, mass - mass[0])

    tt = tt_ref; uu = [uu_ref[:,i] for i in np.arange(uu_ref.shape[1])]
    mass = [np.sum(v) * dx for v in uu]
    plt.plot(tt, mass - mass[0])
    plt.ylim(-3.0e-3, 2.0e-3)
    plt.savefig("../figures/burgers_mass_%s.pdf"%(figname), bbox_inches="tight")


In [ ]:
solve_and_save_burgers(fnum_burgers_EC_diss, "EC_diss")
solve_and_save_burgers(fnum_burgers_central_diss, "central_diss")


# Linear Advection with Inflow Boundary

In [ ]:
N = 200
xx, dx = np.linspace(0, 3, N+1, endpoint=True, retstep=True)
x = xx[:-1]

def ub(t):
    return np.sin(np.pi*t)

def f_linadv(uu):
    t = uu[0]
    u = uu[1:]
    uudot = np.zeros_like(uu)
    uudot[0] = 1
    udot = uudot[1:]
    
    # SBP SAT with interior order of accuracy 2
    udot[0]    = -1.0/dx * (u[1] - u[0]) + 2.0/dx * (ub(t) - u[0])
    udot[1:-1] = -0.5/dx * (u[2:] - u[:-2])
    udot[-1]   = -1.0/dx * (u[-1] - u[-2])
    
    return uudot

def eta_linadv(uu):
    return 0.5 * (0.5*uu[1]*uu[1] + np.dot(uu[2:-1], uu[2:-1]) + 0.5*uu[-1]*uu[-1]) * dx

def deta_linadv(uu):
    w = uu.copy()
    w[0] = 0
    w[1] *= 0.5
    w[-1] *= 0.5
    return w * dx

def mass_linadv(uu):
    return (0.5*uu[1] + np.sum(uu[2:-1]) + 0.5*uu[-1]) * dx

u0 = np.hstack((0, 0*x))
cfl = 0.25
dt = cfl*dx
t_final = 6.

tt0, uu0 = relaxation_ERK(rk.loadRKM("SSP33"), dt, f_linadv, eta_linadv, deta_linadv, u0, 3, 
                          relaxed=False, method="brentq", newdt=True, tol=1.e-14)
t0 = tt0[0]; u0 = uu0[:,0]
t1 = tt0[1]; u1 = uu0[:,1]
t2 = tt0[2]; u2 = uu0[:,2]
t3 = tt0[3]; u3 = uu0[:,3]

time_tmp = time.time()
tt_SSP32_b, uu_SSP32_b, gamma_SSP32_b = cons_or_diss_SSP32(f_linadv, t_final, t0, u0, t1, u1, t2, u2,
        eta=eta_linadv, deta=deta_linadv,
        return_gamma=True, projection=False, relaxation=False, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Baseline scheme:   %.2e s" % time_tmp)

time_tmp = time.time()
tt_SSP32_p, uu_SSP32_p, gamma_SSP32_p = cons_or_diss_SSP32(f_linadv, t_final, t0, u0, t1, u1, t2, u2,
        eta=eta_linadv, deta=deta_linadv,
        return_gamma=True, projection=True, relaxation=False, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Projection scheme: %.2e s" % time_tmp)

time_tmp = time.time()
tt_SSP32_r, uu_SSP32_r, gamma_SSP32_r = cons_or_diss_SSP32(f_linadv, t_final, t0, u0, t1, u1, t2, u2,
        eta=eta_linadv, deta=deta_linadv,
        return_gamma=True, projection=False, relaxation=True, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Relaxation scheme: %.2e s" % time_tmp)

time_tmp = time.time()
dt_ref = 0.01 * dt
tt0, uu0 = relaxation_ERK(rk.loadRKM("SSP33"), dt_ref, f_linadv, eta_linadv, deta_linadv, u0, 3, 
                          relaxed=False, method="brentq", newdt=True, tol=1.e-14)
t0 = tt0[0]; u0 = uu0[:,0]
t1 = tt0[1]; u1 = uu0[:,1]
t2 = tt0[2]; u2 = uu0[:,2]
t3 = tt0[3]; u3 = uu0[:,3]
tt_ref, uu_ref, _ = cons_or_diss_SSP43(f_linadv, t_final, t0, u0, t1, u1, t2, u2, t3, u3,
        eta=eta_linadv, deta=deta_linadv,
        return_gamma=True, projection=False, relaxation=False, adapt_dt=True, adapt_coefficients=True)
time_tmp = time.time() - time_tmp
print("Reference scheme:  %.2e s\n" % time_tmp)


def idx_from_tt(tt):
    idx_min = np.argmax(tt > 2.89)
    idx_max = np.argmax(tt > 3.12)
    return np.arange(idx_min, idx_max)

fig, ax = plt.subplots(1, 1)
tt=tt_SSP32_b; uu=uu_SSP32_b; idx=idx_from_tt(tt)
plt.plot(tt[idx], [eta_linadv(u) for u in np.array(uu)[idx]], label="Baseline SSP($3, 2$)")
tt=tt_SSP32_p; uu=uu_SSP32_p; idx=idx_from_tt(tt)
plt.plot(tt[idx], [eta_linadv(u) for u in np.array(uu)[idx]], label="Projection SSP($3, 2$)")
tt=tt_SSP32_r; uu=uu_SSP32_r; idx=idx_from_tt(tt)
plt.plot(tt[idx], [eta_linadv(u) for u in np.array(uu)[idx]], label="Relaxation SSP($3, 2$)")
tt = tt_ref; uu = uu_ref; idx=idx=idx_from_tt(tt)
plt.plot(tt[idx], [eta_linadv(u) for u in np.array(uu)[idx]], label="Reference")
plt.xlabel(r"$t$"); plt.ylabel(r"Energy"); 
ax.autoscale(enable=True, axis="x", tight=True)
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.savefig("../figures/linear_advection_energy.pdf", bbox_inches="tight")